<a href="https://colab.research.google.com/github/galua2001/colab/blob/main/curve2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
using System.Collections.Generic;
using UnityEngine;

public class Draw : MonoBehaviour
{
    [Header("공유 곡선 (0~1 정규화된 로컬 좌표)")]
    public static List<Vector2> normalizedPoints = new List<Vector2>(); // (0~1) 범위
    public Color curveColor = Color.red;

    [Header("카메라 연결")]
    public Camera mainCamera;

    private static List<Draw> instances = new List<Draw>();
    private LineRenderer line;

    void Awake()
    {
        if (!instances.Contains(this))
            instances.Add(this);

        SetupLineRenderer();
    }

    void Start()
    {
        // 기본 상태에서 점이 없으면 디버그용 기본 점 추가 (메인 컨트롤러인 경우)
        if (normalizedPoints.Count < 2 && IsMainController())
        {
            // 예시: 왼쪽 아래에서 오른쪽 위로 대각선 선분 (정규화 좌표 기준)
            normalizedPoints.Add(new Vector2(0.25f, 0.25f));
            normalizedPoints.Add(new Vector2(0.75f, 0.75f));
        }
        DrawCurve();
    }

    void OnDestroy()
    {
        if (instances.Contains(this))
            instances.Remove(this);
    }

    void Update()
    {
        if (mainCamera == null) return;

        // 왼쪽 클릭: 스프라이트 내부에서만 점 추가 (정규화 좌표가 0~1 범위인지 검사)
        if (Input.GetMouseButtonDown(0))
        {
            Vector2 mouseWorld = mainCamera.ScreenToWorldPoint(Input.mousePosition);
            Vector2 local = transform.InverseTransformPoint(mouseWorld);
            Vector2 normalized = ToNormalized(local);

            // 디버그: 영역 밖이면 로그 출력
            if (normalized.x < 0f || normalized.x > 1f || normalized.y < 0f || normalized.y > 1f)
            {
                Debug.Log("클릭한 좌표가 사각형 영역 밖입니다: " + normalized);
                return;
            }

            if (IsMainController())
            {
                normalizedPoints.Add(normalized);
                UpdateAllCurves();
            }
        }

        // C 키: 메인 컨트롤러에서 곡선 전체 삭제
        if (Input.GetKeyDown(KeyCode.C) && IsMainController())
        {
            normalizedPoints.Clear();
            UpdateAllCurves();
        }

        // 오른쪽 클릭: 스프라이트 내부에서만, 정규화 좌표(0~1)를 통해 사분면 판정 후 조작
        if (Input.GetMouseButtonDown(1))
        {
            Vector2 mouseWorld = mainCamera.ScreenToWorldPoint(Input.mousePosition);
            Vector2 local = transform.InverseTransformPoint(mouseWorld);
            Vector2 normClick = ToNormalized(local);

            // 정규화 좌표가 0~1 범위에 있어야 사각형 내부로 판단
            if (normClick.x < 0f || normClick.x > 1f || normClick.y < 0f || normClick.y > 1f)
            {
                Debug.Log("오른쪽 클릭한 좌표가 사각형 영역 밖입니다: " + normClick);
                return;
            }

            // 정규화 좌표 기준으로 사분면 판정 (중심은 (0.5, 0.5))
            bool isLeft = normClick.x < 0.5f;
            bool isTop = normClick.y > 0.5f;

            if (isLeft && isTop)      Rotate180();     // 왼쪽 위: 180도 회전
            else if (!isLeft && isTop) MirrorY();       // 오른쪽 위: y축 대칭 (x 좌표 반전)
            else if (isLeft && !isTop) ClearCurve();      // 왼쪽 아래: 곡선 삭제
            else                      MirrorDiagonal();  // 오른쪽 아래: y=x 대칭 (x, y 교환)

            UpdateAllCurves();
        }
    }

    // LineRenderer 초기화: 없으면 추가하고 기본 속성 설정
    void SetupLineRenderer()
    {
        line = GetComponent<LineRenderer>();
        if (line == null)
            line = gameObject.AddComponent<LineRenderer>();

        line.startWidth = 0.05f;
        line.endWidth = 0.05f;
        line.material = new Material(Shader.Find("Sprites/Default"));
        // 스프라이트보다 위에 보이도록 sortingOrder를 높임
        line.sortingOrder = 10;
        line.startColor = curveColor;
        line.endColor = curveColor;
        line.useWorldSpace = false; // 로컬 좌표 기준
    }

    // 정규화된 좌표를 이용해 곡선을 그림
    void DrawCurve()
    {
        if (normalizedPoints.Count < 2)
        {
            line.positionCount = 0;
            return;
        }

        line.positionCount = normalizedPoints.Count;
        for (int i = 0; i < normalizedPoints.Count; i++)
        {
            Vector2 local = FromNormalized(normalizedPoints[i]);
            line.SetPosition(i, local);
        }
    }

    // 모든 인스턴스에서 곡선 업데이트 (원본/복제본 동일한 곡선 표시)
    void UpdateAllCurves()
    {
        foreach (var instance in instances)
        {
            instance.DrawCurve();
        }
    }

    // 로컬 좌표 -> (0~1) 정규화 (스프라이트 내부 기준: 중앙이 (0.5, 0.5))
    Vector2 ToNormalized(Vector2 localPos)
    {
        Vector2 size = GetSize();
        return new Vector2((localPos.x + size.x / 2f) / size.x, (localPos.y + size.y / 2f) / size.y);
    }

    // (0~1) 정규화 좌표 -> 실제 로컬 좌표로 환산
    Vector2 FromNormalized(Vector2 norm)
    {
        Vector2 size = GetSize();
        return new Vector2(norm.x * size.x - size.x / 2f, norm.y * size.y - size.y / 2f);
    }

    // 스프라이트의 크기를 가져옴 (SpriteRenderer 사용; 없으면 (1,1) 반환)
    Vector2 GetSize()
    {
        var sprite = GetComponent<SpriteRenderer>();
        return sprite != null ? sprite.bounds.size : Vector2.one;
    }

    // 오른쪽 클릭 시 조작 기능들

    // 180도 회전: 각 정규화 좌표를 반전 (중심 기준)
    void Rotate180()
    {
        for (int i = 0; i < normalizedPoints.Count; i++)
        {
            normalizedPoints[i] = -normalizedPoints[i] + Vector2.one;
        }
    }

    // y축 대칭: x 좌표를 1 - x로 변경
    void MirrorY()
    {
        for (int i = 0; i < normalizedPoints.Count; i++)
        {
            Vector2 p = normalizedPoints[i];
            normalizedPoints[i] = new Vector2(1f - p.x, p.y);
        }
    }

    // 곡선 삭제: 모든 점 삭제
    void ClearCurve()
    {
        normalizedPoints.Clear();
    }

    // y=x 대칭: x와 y 값을 서로 교환
    void MirrorDiagonal()
    {
        for (int i = 0; i < normalizedPoints.Count; i++)
        {
            Vector2 p = normalizedPoints[i];
            normalizedPoints[i] = new Vector2(p.y, p.x);
        }
    }

    // 첫 번째 인스턴스(원본)를 메인 컨트롤러로 지정하여 입력 처리를 위임
    bool IsMainController()
    {
        return instances.Count > 0 && instances[0] == this;
    }
}